In [1]:
pip install afinn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import json
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from afinn import Afinn
from transformers import pipeline
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
!python -V


Python 3.11.7


In [ ]:
data=pd.read_csv(r'/kaggle/input/spicy-dataset/Emoji_Dataset.csv')
with open('/kaggle/input/spicy-dataset/Sentiment Group.json', 'r') as file:
    groups=json.load(file)

In [ ]:
path='https://tfhub.dev/google/elmo/3'
def Embedding(sentence, batch_size=5000, model_path=path, key='elmo'):
    tf_sent=tf.stack([tf.constant(x) for x in sentence], axis=0)
    embed= hub.KerasLayer(model_path, output_key=key, trainable=False)
    result=[]
    count=1
    for start in range(0, len(tf_sent), batch_size):
        end=start+batch_size  
        embed_sent=embed(tf_sent[start:end])
        result.extend(embed_sent)
        print(f"embedding sentences batch{count} done batch{count+1} processing......")
        count+=1
    return tf.stack(result) 

In [5]:
def classify(sentence):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    afn=Afinn()
    score=afn.score(sentence)
    sentiment= 1 if score > 0 else 1 if score < 0 else 2
    if sentiment== 2:
        return sentiment
    if sentiment == 1:
        sentiment_analyzer = pipeline("sentiment-analysis", model=model_name, device=0)
        label=sentiment_analyzer(sentence)[0]['label']
        sentiment= 1 if label =='POSITIVE' else 0
    return sentiment


In [6]:
positive=pd.DataFrame(groups['POSITIVE'])
negative=pd.DataFrame(groups['NEGATIVE'])
neutral=pd.DataFrame(groups['NEUTRAL'])
object_=pd.DataFrame(groups['OBJECT'])
object_.head()

,emoji,name
0,🤩,star-struck
1,💀,skull
2,👹,ogre
3,👺,goblin
4,👻,ghost


In [7]:
def single_embed(sentence):
    new=[]
    for sent in sentence:
        n_sent=sent.split()
        while len(n_sent)>6:
            sen=n_sent[:6]
            new.append(' '.join(sen))
            n_sent=sent[6:]
        if len(n_sent)!=0:
            new.append(' '.join(n_sent))
    embed=Embedding(new)
    result=[]
    for em in embed:
        if em.shape[0]!=6:
            p=6-em.shape[0]
            pad_em=np.pad(em,((0,p),(0,0)))
            result.append(tf.constant(pad_em))
        else:
            result.append(em)
            
    return tf.stack(result)
          

In [8]:
pos_arr=single_embed(list(positive['name'])).numpy()
neg_arr=single_embed(list(negative['name'])).numpy()
neu_arr=single_embed(list(neutral['name'])).numpy()
neu_arr.shape

embedding sentences batch1 done batch2 processing......
embedding sentences batch1 done batch2 processing......
embedding sentences batch1 done batch2 processing......


(58, 6, 1024)

In [9]:
import pickle
standard_array={'pos_arr':pos_arr, 'neg_arr':neg_arr, 'neu_arr':neu_arr}


with open('Standard Array.pkl', 'wb') as file:
    pickle.dump(standard_array, file)

In [10]:
obj_dic={name:emoji for name, emoji in zip(list(object_['name']),list(object_['emoji']))}
pos_dic={idx:emoji for idx, emoji in enumerate(list(positive['emoji']))}
neg_dic={idx:emoji for idx, emoji in enumerate(list(negative['emoji']))}
neu_dic={idx:emoji for idx, emoji in enumerate(list(neutral['emoji']))}
decode={0:'Negative', 1:'Positive', 2:'Neutral'}

Ref_dic={'obj_dic':obj_dic, 'pos_dic':pos_dic, 'neg_dic':neg_dic, 'neu_dic':neu_dic, 'decode':decode}

with open('Ref_dic.pkl', 'wb') as file:
    pickle.dump(Ref_dic, file)

In [11]:
def extract(sent_arr, group):
    if group==0:
        emo= np.argsort(euclidean_distances(neg_arr.reshape(neg_arr.shape[0],-1), 
                                          sent_arr.reshape(1,-1)), axis=0).reshape(1,-1)[0][:20]
        emoji=neg_dic[np.random.choice(emo)]
    elif group==1:
        emo= np.argsort(euclidean_distances(pos_arr.reshape(pos_arr.shape[0], -1),
                                          sent_arr.reshape(1,-1)), axis=0).reshape(1,-1)[0][:20]
        emoji=pos_dic[np.random.choice(emo)]
        
    else:
        emo=np.argsort(euclidean_distances(neu_arr.reshape(neu_arr.shape[0], -1),
                                        sent_arr.reshape(1,-1)), axis=0).reshape(1,-1)[0][:20]
        emoji=neu_dic[np.random.choice(emo)]        
    
    return emoji


def predict(sentences):
    prediction=[]
    for sentence in sentences:
        sent=sentence.split()
        work=[]
        while len(sent)>6:
            wrk=sent[:6]
            work.append(' '.join(wrk))
            sent=sent[6:]
        if sent!=0:
            work.append(' '.join(sent))
    
        embed=Embedding(work).numpy()
        if embed.shape[1]!=6:
            p=6-embed.shape[1]
            embed=np.pad(embed,((0,0),(0,p),(0,0)))
            
        grp=[classify(x) for x in work]
        grp_emo=[extract(arr,group) for arr,group in zip(embed,grp)]
        result=' '.join([x+' '+re.sub(r'\u200d','',y) for x,y in zip(work,grp_emo)]).split()
        
        
        for i,x in enumerate(result):
            if x in obj_dic.keys():
                result[i]=x+obj_dic[x]
            else:
                pass
        
        prediction.append([' '.join(result), decode[grp[0]]])
    return  prediction, 
    

In [1]:
text="is a powerful and intuitive tool designed to enhance your text with the perfect emojis. Whether you're looking to add a touch of emotion, humor, or emphasis, TextEmojiFusion is not just a tool; it's your creative partner in the digital realm, transforming mundane messages into vibrant conversations. Imagine a platform that understands the nuances of your emotions and the subtleties of your humor, then curates a selection of emojis that perfectly align with your intent. TextEmojiFusion does just that, bridging the gap between plain text and the rich expressiveness of emojis. With its intuitive design, it's incredibly user-friendly, inviting you to elevate your communication effortlessly. Whether you're conveying heartfelt sentiments, injecting a dose of wit, or emphasizing key points, TextEmojiFusion integrates emojis with such finesse that your messages will capture hearts and imaginations alike. It's not merely about enhancing text; it's about enriching connections and sharing experiences that words alone can't fully express. With TextEmojiFusion, every message becomes a canvas, and every conversation is an opportunity to create something memorable. So why settle for ordinary when you can communicate with color, emotion, and life? Embrace the power of TextEmojiFusion and watch your digital interactions blossom into engaging, expressive, and extraordinary exchanges."
text

"is a powerful and intuitive tool designed to enhance your text with the perfect emojis. Whether you're looking to add a touch of emotion, humor, or emphasis, TextEmojiFusion is not just a tool; it's your creative partner in the digital realm, transforming mundane messages into vibrant conversations. Imagine a platform that understands the nuances of your emotions and the subtleties of your humor, then curates a selection of emojis that perfectly align with your intent. TextEmojiFusion does just that, bridging the gap between plain text and the rich expressiveness of emojis. With its intuitive design, it's incredibly user-friendly, inviting you to elevate your communication effortlessly. Whether you're conveying heartfelt sentiments, injecting a dose of wit, or emphasizing key points, TextEmojiFusion integrates emojis with such finesse that your messages will capture hearts and imaginations alike. It's not merely about enhancing text; it's about enriching connections and sharing experi

In [12]:
spicy=predict([text])
spicy

embedding sentences batch1 done batch2 processing......


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

([["is a powerful and intuitive tool 🙃 designed to enhance your text with 🙊 the perfect emojis. Whether you're looking 💌 to add a touch of emotion, 💭 humor, or emphasis, TextEmojiFusion is not 🤤 just a tool; it's your creative 🙃 partner in the digital realm, transforming 😺 mundane messages into vibrant conversations. Imagine 🤗 a platform that understands the nuances 🧔 of your emotions and the subtleties 🦾 of your humor, then curates a 😗 selection of emojis that perfectly align 😗 with your intent. TextEmojiFusion does just 🤏 that, bridging the gap between plain 🤞 text and the rich expressiveness of 😀 emojis. With its intuitive design, it's 😽 incredibly user-friendly, inviting you to elevate ☺ your communication effortlessly. Whether you're conveying 🤓 heartfelt sentiments, injecting a dose of 😉 wit, or emphasizing key🔑 points, TextEmojiFusion 🙈 integrates emojis with such finesse that 🙏 your messages will capture hearts and 👾 imaginations alike. It's not merely about 😺 enhancing text; i